# How to use easy campaign from python


## RBFE

### SETUP

In [2]:
# Prepare Setup
from openff.units import unit
from rdkit import Chem
from openfe import SmallMoleculeComponent, ProteinComponent, SolventComponent

suppl = Chem.SDMolSupplier("molecules/rbfe/p38_old_ligands.sdf")
ligands = [SmallMoleculeComponent.from_rdkit(m) for m in suppl]
protein = ProteinComponent.from_pdb_file("molecules/rbfe/p38_old_protein.pdb")
solvent = SolventComponent(ion_concentration=0.15*unit.molar)

# Prepare setup Planning
from openfe import setup
from openfe.protocols.openmm_rbfe import RelativeLigandProtocol

atom_mapper = setup.LomapAtomMapper()
atom_mapping_scorer = setup.lomap_scorers.default_lomap_score
ligand_network_planner = setup.MinimalSpanningNetworkPlanner(mappers=[atom_mapper], mapping_scorer=atom_mapping_scorer)

rbfe_protocol = RelativeLigandProtocol(settings=RelativeLigandProtocol._default_settings())
alchemical_network_planner = setup.RBFEAlchemicalNetworkPlanner(name="easy_rbfe_p38", ligand_network_planner=ligand_network_planner, protocol=rbfe_protocol)

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [3]:
# Plan Calculations
# This might take up to 10 minutes for the example system.

alchemical_network = alchemical_network_planner(ligands=ligands, protein=protein, solvent=solvent)
alchemical_network

<AlchemicalNetwork-22eb937814faf0dbb28235012c6be3ba>

In [5]:
print("Transformations to calculate:", len(alchemical_network.edges))
for edge in alchemical_network.edges:
    print("\t",edge.name)

Transformations to calculate: 66
	 easy_rbfe_p38_lig_p38a_2aa_receptor_lig_p38a_2z_receptor
	 easy_rbfe_p38_lig_p38a_2c_receptor_lig_p38a_3flz_receptor
	 easy_rbfe_p38_lig_p38a_2l_solvent_lig_p38a_3fln_solvent
	 easy_rbfe_p38_lig_p38a_2ff_solvent_lig_p38a_3fln_solvent
	 easy_rbfe_p38_lig_p38a_2g_receptor_lig_p38a_3flz_receptor
	 easy_rbfe_p38_lig_p38a_2n_solvent_lig_p38a_2p_solvent
	 easy_rbfe_p38_lig_p38a_2aa_solvent_lig_p38a_3fly_solvent
	 easy_rbfe_p38_lig_p38a_2v_receptor_lig_p38a_3fln_receptor
	 easy_rbfe_p38_lig_p38a_3flw_receptor_lig_p38a_3fly_receptor
	 easy_rbfe_p38_lig_p38a_2e_solvent_lig_p38a_3fln_solvent
	 easy_rbfe_p38_lig_p38a_2g_solvent_lig_p38a_3flz_solvent
	 easy_rbfe_p38_lig_p38a_2e_receptor_lig_p38a_3fln_receptor
	 easy_rbfe_p38_lig_p38a_2n_receptor_lig_p38a_2s_receptor
	 easy_rbfe_p38_lig_p38a_2bb_receptor_lig_p38a_3flq_receptor
	 easy_rbfe_p38_lig_p38a_2f_solvent_lig_p38a_3flz_solvent
	 easy_rbfe_p38_lig_p38a_2bb_solvent_lig_p38a_3flq_solvent
	 easy_rbfe_p38_lig_p3

### Calculations
in this stage the planned transfromations are going to be calculated.

**WARNING:** These scripts shall serve as examples. for RBFE the calculations are taking very long and you should have HPC environment where you can parallelize the execution. - here in the future our Exorcist Project will help you

In [ ]:
# Output:
output_dir = "output"   

In [ ]:
from gufe.protocols.protocoldag import execute_DAG

transformation_results = []
for transformation in alchemical_network.edges:
    
    print("Running the simulations: ", transformation.name)
    transformation_task_dag = transformation.create()
    transformation_task_result = execute_DAG(transformation_task_dag)
    transformation_result = transformation.protocol.gather([transformation_task_result])
    transformation_results.append(transformation_result)
print("Done")

### Gather results:

finally we want to gather the results, to see what we calculated.

In [ ]:
for transformation_result in transformation_results:
    if transformation_result.ok():
        estimate = transformation_result.get_estimate()
        uncertainty = transformation_result.get_uncertainty()
    else:
        estimate = uncertainty = None
    print("Estimate: ", transformation_result.name, estimate, uncertainty)